In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as reg
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor



# Loading the data

In [2]:
data=pd.read_csv('concrete_data.csv')


IncompleteRead: IncompleteRead(116192 bytes read)

In [ ]:
dir(data)

# Exploration with pandas

In [ ]:
type(data)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.count()

## Cleaning data - checking for missing and duplicate values

In [ ]:
data.isnull().values.any()

In [ ]:
data.isnull().any()

In [ ]:
 data[data.duplicated()]

In [ ]:
data.drop_duplicates(keep="first",inplace=True)

In [ ]:
data

In [ ]:
data.info()

# Visualizing and understanding the data

In [ ]:
data.describe()

###### check out the strength for concrete at different ages

In [ ]:
data.groupby('age')[['strength']].max()

In [ ]:
data.groupby('cement')[['strength']].max()

#### strength when age is 1

In [ ]:
data[data['age']==1][['strength']].max()

In [ ]:
data[data['strength']>80].shape[0]

#### strength when age is 365

In [ ]:
data.loc[data['age']==365][['strength']].max()

#### strength when cement is 540.000000 kg(max)

In [ ]:
data.loc[data['cement']==540.000000][['strength']].max()

##### we can see that we get a good strength if we use more cement

#### strength when cement is 102.000000 (min)

In [ ]:
data.loc[data['cement']==102.000000][['strength']].max()

##### we can see that we get a poor strength if we use more cement

#### strength when cement is max and age is low

In [ ]:
data.loc[(data['cement']==540.000000) & (data['age']<30)][['strength']].max()

#### strength when cement is max and age is high

In [ ]:
data.loc[(data['cement']==540.000000) & (data['age']>200)][['strength']].max()

#### stength when cement is min and age is high

In [ ]:
data.loc[(data['cement']==102.000000) & (data['age']>80)][['strength']].max()

#### strength when cement is min and age is low

In [ ]:
data.loc[(data['cement']==102.000000) & (data['age']<30)][['strength']].max()

#### observation 1- regardless of the ages of cement the amount still determines it strength. if its in low amount the concrete yields lower strength and vice versa

#### observation 2- no cement has lasted more than 3 months

### Plotting graphs

In [ ]:

#data['strength'].plot(kind="hist",bins=80,color='red')
# plt.hist(data['strength'],kind="hist",bins=80,rwidth=0.5)
plt.hist(data['strength'],bins=10,rwidth=0.5,ec='black',color="red")

In [ ]:
data['strength'].mode()

#### very few concretes are extremely strong from the data set since only few are above 80; 3 to be exact. while most are beloe average 31

In [ ]:
plt.hist(data['slag'],bins=10,rwidth=0.5,ec='black',color="purple")

In [ ]:
data['slag'].mode()

#### for most slag was not even included

In [ ]:
plt.hist(data['cement'],bins=10,rwidth=0.5,ec='black',color="skyblue")

In [ ]:
data['cement'].mode()

In [ ]:

plt.hist(data['superplasticizer'],bins=10,rwidth=0.5,ec='black',color="blue")

### Checking correlation of the colums with strength using heatmap

In [ ]:
plt.figure(figsize=[10,10])
sns.heatmap(data=data.corr(),annot=True,cmap="Blues")

In [ ]:
strength_cement_corr = round(data['cement'].corr(data['strength']), 3)

In [ ]:

plt.figure(figsize=[15,10])

plt.scatter(x=data['cement'],y=data['strength'],color='green')
plt.title(f'Strength vs Cement (kg) corr {strength_cement_corr}')

In [ ]:
sns.pairplot(data,kind='reg')

## Split data into train and test

In [ ]:
target=data['strength']
features=data.drop('strength',axis=1)

In [ ]:
features

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=10)

In [ ]:
X_train.shape[0]

In [ ]:
X_test.shape[0]

## Multivariable Regression

In [ ]:
model=reg()
model.fit(X_train,y_train)
pd.DataFrame(data=model.coef_,columns=['coef'],index=X_train.columns)

In [ ]:
model.intercept_

### using statsmodel

In [ ]:
# Original Model
X_incl_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_incl_const)
results = model.fit()
corr = round(y_train.corr(results.fittedvalues), 2)
plt.figure(figsize=[10,10])
plt.title(f'Predicted concrete strength vs Actual concrete strength with correlation {corr}')
plt.scatter(x=y_train,y=results.fittedvalues,color='pink')
plt.xlabel('Actual concrete strength')
plt.ylabel('Predicted values concrete strength')
plt.plot(y_train,y_train,color='black')

#### Residual plot - checking for Normality

In [ ]:
# plt.figure(figsize=[10,10])
plt.title(f'Residuals of concrete strength vs Predicted concrete strength with correlation ')
plt.scatter(x=results.fittedvalues,y=results.resid,color='skyblue')
plt.xlabel('Predicted concrete strength')
plt.ylabel('Residuals of the concrete strength')

In [ ]:
plt.figure(figsize=[10,10])

resid_skew=round(results.resid.skew())
resid_mean=round(results.resid.mean())
sns.displot(results.resid)
plt.title(f'Distribution plot of the residuals mean: {resid_mean}, skew {resid_skew}',fontsize=18)

In [ ]:
pd.DataFrame(data=results.resid,columns=['residuals'])


In [ ]:
mse=round(results.mse_resid,3)
r_squared=round(results.rsquared,3)
print('mean squared error',mse)
print('r squared',r_squared)


## Data Transformation

In [ ]:
data['strength'].skew()

In [ ]:
# Model with omitted features

target_1=np.log(data['strength'])
features_1=data.drop(['strength',
                      'fly_ash'
                      ,
                      'coarse_aggregate',
                     'fine_aggregate'
                     ],axis=1)
X_train,X_test,y_train,y_test=train_test_split(features_1,target_1,test_size=0.2,random_state=10)
model_1=reg()
model_1.fit(X_train,y_train)
pd.DataFrame(data=model_1.coef_,columns=['coef'],index=X_train.columns)



In [ ]:
# Original Model
X_incl_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_incl_const)
results = model.fit()
corr = round(y_train.corr(results.fittedvalues), 2)
plt.figure(figsize=[10,10])
plt.title(f'Predicted concrete strength vs Actual concrete strength with correlation {corr}')
plt.scatter(x=y_train,y=results.fittedvalues,color='pink')
plt.xlabel('Actual concrete strength')
plt.ylabel('Predicted values concrete strength')
plt.plot(y_train,y_train,color='black')

In [ ]:
# plt.figure(figsize=[10,10])
plt.title(f'Residuals of concrete strength vs Predicted concrete strength with correlation ')
plt.scatter(x=results.fittedvalues,y=results.resid,color='skyblue')
plt.xlabel('Predicted concrete strength')
plt.ylabel('Residuals of the concrete strength')



In [ ]:
plt.figure(figsize=[10,10])

resid_skew=round(results.resid.skew())
resid_mean=round(results.resid.mean())
sns.displot(results.resid)
plt.title(f'Distribution plot of the residuals mean: {resid_mean}, skew {resid_skew}',fontsize=18)

In [ ]:
mse=round(results.mse_resid,3)
r_squared=round(results.rsquared,3)
print('mean squared error',mse)
print('r squared',r_squared)


In [ ]:
# Using log

target_2=np.log(data['strength'])
features=data.drop('strength',axis=1)
X_train,X_test,y_train,y_test=train_test_split(features_2,target_2,test_size=0.2,random_state=10)
model_2=reg()
model_2.fit(X_train,y_train)
pd.DataFrame(data=model_2.coef_,columns=['coef'],index=X_train.columns)



In [ ]:
X_incl_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_incl_const)
results = model.fit()
corr = round(y_train.corr(results.fittedvalues), 2)
plt.figure(figsize=[10,10])
plt.title(f'Predicted concrete strength vs Actual concrete strength with correlation {corr}')
plt.scatter(x=y_train,y=results.fittedvalues,color='pink')
plt.xlabel('Actual concrete strength')
plt.ylabel('Predicted values concrete strength')
plt.plot(y_train,y_train,color='black')



In [ ]:
plt.figure(figsize=[10,10])
plt.title(f'Residuals of concrete strength vs Predicted concrete strength with correlation ')
plt.scatter(x=results.fittedvalues,y=results.resid,color='skyblue')
plt.xlabel('Predicted concrete strength')
plt.ylabel('Residuals of the concrete strength')

In [ ]:
plt.figure(figsize=[10,10])

resid_skew=round(results.resid.skew())
resid_mean=round(results.resid.mean())
sns.displot(results.resid)
plt.title(f'Distribution plot of the residuals mean: {resid_mean}, skew {resid_skew}',fontsize=18)

In [ ]:
mse=round(results.mse_resid,3)
r_squared=round(results.rsquared,3)
print('mean squared error',mse)
print('r squared',r_squared)

## log scaling is only good when the residual distribution is not normal